In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets
import plotly.express as px
from faker import Faker

# Dashboard Title
print("### Digital Twin: Aeronautical Sensor Manufacturing ###")

# Configurar Faker
faker = Faker()

# Generar una lista limitada de técnicos
technicians = [faker.first_name() for _ in range(12)]

# Cargar los datos simulados
data = pd.read_csv("C:/Digital_twins/data/large_simulated_equipment_data_with_centrals.csv")

# Reemplazar los técnicos en el dataset por la lista limitada
data['Technician'] = [technicians[i % len(technicians)] for i in range(len(data))]

# Función para filtrar y analizar datos
def filter_and_analyze(equipment=None, step=None, technician=None, month=None, sensor_type=None):
    filtered_data = data.copy()

    # Aplicar filtros según los valores seleccionados
    if equipment:
        filtered_data = filtered_data[filtered_data["Equipment"] == equipment]
    if step:
        filtered_data = filtered_data[filtered_data["Step"] == step]
    if technician:
        filtered_data = filtered_data[filtered_data["Technician"] == technician]
    if month:
        filtered_data = filtered_data[filtered_data["Month"] == month]
    if sensor_type:
        filtered_data = filtered_data[filtered_data["Type"] == sensor_type]

    # Validar si hay datos
    if filtered_data.empty:
        print("No data available for the selected filters.")
        return

    # Tabla de datos filtrados
    print("Filtered Data:")
    display(filtered_data.head(10))

    # Gráfico: Comparación de tiempos estimados vs reales
    plt.figure(figsize=(10, 6))
    filtered_data_melted = filtered_data.melt(
        id_vars=["Step"], 
        value_vars=["Estimated_Time", "Real_Time"], 
        var_name="Time_Type", 
        value_name="Time"
    )
    sns.barplot(data=filtered_data_melted, x="Step", y="Time", hue="Time_Type", dodge=True, palette="Set2")
    plt.title("Real vs Estimated Time by Step")
    plt.xlabel("Step")
    plt.ylabel("Time (minutes)")
    plt.xticks(rotation=45)
    plt.legend(title="Time Type")
    plt.show()

    # Gráfico: Resultados agregados (Pass vs Fail)
    plt.figure(figsize=(8, 5))
    sns.countplot(data=filtered_data, x="Result", hue="Type")
    plt.title("Result Distribution by Sensor Type")
    plt.xlabel("Result")
    plt.ylabel("Count")
    plt.legend(title="Sensor Type")
    plt.show()

    # Producción por mes (Gráfico interactivo con Plotly)
    month_data = filtered_data.groupby("Month")[["Equipment"]].nunique().reset_index()
    month_data.columns = ["Month", "Number of Equipment"]

    fig = px.bar(month_data, x="Month", y="Number of Equipment", 
                 title="Production by Month", 
                 labels={"Number of Equipment": "Number of Equipment Produced"},
                 hover_data={"Month": True, "Number of Equipment": True})
    fig.update_layout(xaxis_title="Month", yaxis_title="Number of Equipment Produced")
    fig.show()

# Widgets interactivos
equipment_widget = widgets.Dropdown(
    options=[None] + list(data["Equipment"].unique()),
    description="Equipment:",
    style={"description_width": "initial"}
)

step_widget = widgets.Dropdown(
    options=[None] + list(data["Step"].unique()),
    description="Step:",
    style={"description_width": "initial"}
)

technician_widget = widgets.Dropdown(
    options=[None] + technicians,
    description="Technician:",
    style={"description_width": "initial"}
)

month_widget = widgets.Dropdown(
    options=[None] + list(data["Month"].unique()),
    description="Month:",
    style={"description_width": "initial"}
)

sensor_type_widget = widgets.Dropdown(
    options=[None] + list(data["Type"].unique()),
    description="Sensor Type:",
    style={"description_width": "initial"}
)

# Interactividad
interact(filter_and_analyze, 
         equipment=equipment_widget, 
         step=step_widget, 
         technician=technician_widget, 
         month=month_widget, 
         sensor_type=sensor_type_widget)


### Digital Twin: Aeronautical Sensor Manufacturing ###


interactive(children=(Dropdown(description='Equipment:', options=(None, 'Equipment_1', 'Equipment_2', 'Equipme…

<function __main__.filter_and_analyze(equipment=None, step=None, technician=None, month=None, sensor_type=None)>